<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [ ]:
   "source": [
    "from src.model import *\n",
    "# In TensorFlow a device name is a string of the following form:\n",
    "#   /job:<name>/replica:<replica>/task:<task>/device:<type>:<device_num>\n",
    "#\n",
    "# <name> is a short identifier conforming to the regexp\n",
    "#     [a-zA-Z][_a-zA-Z]*\n",
    "# <type> is a supported device type (e.g. 'cpu' or 'gpu')\n",
    "# <replica>, <task>, <device_num> are small non-negative integers and are\n",
    "# densely allocated (except in tests).\n",
    "#\n",
    "# For some purposes, we also allow device patterns, which can specify\n",
    "# some or none of the specific fields above, with missing components,\n",
    "# or \"<component>:*\" indicating \"any value allowed for that component.\n",
    "#\n",
    "# For example:\n",
    "#   \"/job:param_server\"   - Consider any devices in the \"param_server\" job\n",
    "#   \"/device:cpu:*\"       - Consider any cpu devices in any job/task/replica\n",
    "#   \"/job:*/replica:*/task:*/device:cpu:*\"  - Consider any cpu devices in any\n",
    "#                                             job/task/replica\n",
    "#   \"/job:w/replica:0/task:0/device:gpu:*\"  - Consider any gpu devices in\n",
    "#                                             replica 0, task 0, of job \"w\""
   ]
  },

In [ ]:
    "dataload = Sass('/home/penalvad/stattus4/test/', num_samples=5, number_of_batches=1, split_tax=.9, freq_size=480, time_size=640, same_data_validation=True)\n",
    "dataload_2 = Sass('/home/penalvad/stattus4/benchfull/', num_samples=7, number_of_batches=1, split_tax=.9, freq_size=600, time_size=80, same_data_validation=True)"

In [ ]:
    "dtrain = dataload.training()\n",
    "dtrain_2 = dataload_2.training()"

In [ ]:

    "dtrain = dataload.training()\n",
    "dtest = dataload.testing()\n",
    "\n",
    "data_train = np.ndarray( shape=(14,600,80,3) )\n",
    "data_test = np.ndarray( shape=(8,600,50,3) )\n",
    "\n",
    "for i in range(6):\n",
    "    \n",
    "  data_train[2*i,:,:,:] = dtrain[i][0][2]\n",
    "  data_train[2*i+1,:,:,:] = dtrain[i][1][2]\n",
    "\n",
    "#for i in range(4):\n",
    "\n",
    "  #data_test[2*i,:,:,0] = dtest[i][0][2]\n",
    "  #data_test[2*i+1,:,:,0] = dtest[i][1][2]"

In [ ]:
    "# Architecture Basics"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "colab_type": "text",
    "id": "99BygeASuBJD"
   },
   "source": [
    "##  Gated CNN\n",
    "  Gated CNN is a doubled CNN in whom one of the convoluted signals does the role of opening/closing the network, giving an **Attention Mechanism** to the convolution, for being activated by a sigmoid.\n",
    "  It gives non-vanishing gradient, since the multiplication rule for the derivative applies, also, applies gradient to the linear convoluted part.\n",
    "  "

In [ ]:
    "### GCNN 2D\n",
    "  This unit uses sigmoid function in all frame pixels and do element-wise multiplication. Use it for images or time-series TF Features frame. The standard convolution mode is Valid. The output size of the convolution is given by:\n",
    " \n",
    " Padding Valid\n",
    "\\begin{align}\n",
    "dimemsion output size = \\ceil{ \\frac{dim size - (kernel dim size - 1)*dilation rate}{stride} }\n",
    "\\end{align} \n",
    "\n",
    "Where dilation rate is how may pixel are each filter spaced from each other, when dilation rate >= 1 then stride = 1 (since if you stride will be losing information from the data)."

In [ ]:
    "#Cascading deepness networks mem size\n",
    "    \n",
    "def channeling_rule(channelin, filterlist):        \n",
    "  channellist = [channelin]\n",
    "  for filter in filterlist:\n",
    "    channellist.append(channellist[-1] + filter + 3)  \n",
    "\n",
    "  return channellist "
   ]

In [ ]:
    "# Label Vector Feed\n",
    "# dtrain[0][0][1]\n",
    "ltrain = []\n",
    "ltest = []\n",
    "\n",
    "for i in range(14):    \n",
    "  ltrain.append([0,1])   \n",
    "  ltrain.append([1,0])\n",
    "    \n",
    "#for i in range(4):\n",
    "  #ltest.append([0,1])    \n",
    "  #ltest.append([1,0])"
   ]

In [ ]:
 "for i in range(10):  \n",
    "  feed_dict[buildgraph.signal_in[i]] = data_train[:14,:,8*i:8*(i+1),:]\n",
    "\n",
    "feed_dict[buildgraph.label_tensor] = ltrain[:14]\n",
    "feed_dict[buildgraph.learning_rate] = 0.00001 \n",
    "\n",
    "output = buildgraph.run_cgraph(feed_dict, op_to_run = [buildgraph.namescopes['softmax'][-1].name, buildgraph.namescopes['softmax_3'][-1].name, buildgraph.namescopes['softmax_2'][-1].name, buildgraph.namescopes['softmax_4'][-1].name, buildgraph.namescopes['softmax_5'][-1].name, buildgraph.namescopes['softmax_6'][-1].name], number_of_runs = 40, mode = 'minimize', ckpt_dir_name='../ckpt/model', new_session=True, output_log = True, stop=0.4, adaptative_lr=True, verbose=True)\n",
    "\n",
    "#for i in range(10):  \n",
    "  #feed_dict[buildgraph.signal_in[i]] = data_train[:,:,5*i:5*(i+1),:]\n",
    "\n",
    "#feed_dict[buildgraph.l_vec] = ltrain[:14]\n",
    "#feed_dict[buildgraph.learning_rate] = 0.00001 \n",
    "\n",
    "#acc = buildgraph.run_cgraph(feed_dict, op_to_run = buildgraph.namescopes['reducemean'][-1].name, number_of_runs = 1, mode = 'x', new_session=True, output_log = True, auto_lr = False, verbose=True)\n",
    "\n",
    "#print('Tensor {}'.format(buildgraph.namescopes.gcnn2d_1[1].name))\n"

In [ ]:
# Example of Recovering Model from .meta and .data. It uses .index and checkpoint to map variables and last checkpoint to the graph \n",
graph = tf.Graph()
    
with graph.as_default():
    
    saver = tf.train.import_meta_graph(\"../ckpt/model.meta\", import_scope='')
    feed_dict = {}
    with tf.Session(graph=graph) as sess:
    Restore variables from disk.
    "    \n",
    "    saver.restore(sess, tf.train.latest_checkpoint('../ckpt/') )    \n",
    "    feed_dict[sess.graph.get_tensor_by_name('signal_in:0')] = data_train[:14,:,:8,:]\n",
    "    for i in np.arange(1,10):  \n",
    "      feed_dict[sess.graph.get_tensor_by_name('signal_in_'+str(i)+':0')] = data_train[:14,:,8*i:8*(i+1),:]\n",
    "    feed_dict[sess.graph.get_tensor_by_name('losscrossentropy/labels:0')] = ltrain[:14]\n",
    "    feed_dict[sess.graph.get_tensor_by_name('learning_rate:0')] = 0.0001      \n",
    "  #for op in graph.get_operations():\n",
    "    #print(2*'\\n')\n",
    "    #print(op.name)  \n",
    "    minimize_op = sess.graph.get_operation_by_name('losscrossentropy/Adam')\n",
    "    loss = sess.graph.get_tensor_by_name('losscrossentropy/categorical_crossentropy/weighted_loss/value:0')\n",
    "    acc = sess.graph.get_tensor_by_name('reducemean/Mean:0')\n",
    "    for i in range(400):\n",
    "      lossval,_ = sess.run([loss,minimize_op], feed_dict=feed_dict)\n",
    "      accval = sess.run([acc], feed_dict=feed_dict)\n",
          if lossval < 0.3:\n",
            saver.save(sess, '../ckpt/model2')\n",
            print('\\n')\n",
            print('saving')\n",
            print('\\n')\n",
            \n",
          print('loss ',lossval)     \n",
          print('acc ', accval)\n",
          print(2*'\\n')\n",
    
